# Lesson 3: Training Models in Low Carbon Regions

* Import libraries to train a model locally.

In [1]:
import numpy as np
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2024-08-20 09:51:24.535423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-20 09:51:25.045208: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-20 09:51:25.159448: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-20 09:51:25.159473: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

* Create a dataset.

This is an example from the [Advanced Learning Algorithms](https://www.coursera.org/learn/advanced-learning-algorithms) course.

In [2]:
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, 
    centers=centers, 
    cluster_std=std,
    random_state=30)

* Create the model.

In [3]:
# Create the model
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

2024-08-20 09:52:21.107282: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-08-20 09:52:21.108032: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-08-20 09:52:21.108063: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-30-113-97.us-west-1.compute.internal): /proc/driver/nvidia/version does not exist
2024-08-20 09:52:21.110501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

* Train the model.

In [5]:
model.fit(
    X_train,y_train,
    epochs=200
)

Epoch 1/200
4/4 [==============================] - 0s 4ms/step - loss: 2.7091
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 2.4599
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 2.2325
Epoch 4/200
4/4 [==============================] - 0s 1ms/step - loss: 2.0310
Epoch 5/200
4/4 [==============================] - 0s 1ms/step - loss: 1.8522
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 1.6948
Epoch 7/200
4/4 [==============================] - 0s 1ms/step - loss: 1.5560
Epoch 8/200
4/4 [==============================] - 0s 1ms/step - loss: 1.4329
Epoch 9/200
4/4 [==============================] - 0s 1ms/step - loss: 1.3250
Epoch 10/200
4/4 [==============================] - 0s 1ms/step - loss: 1.2329
Epoch 11/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1537
Epoch 12/200
4/4 [==============================] - 0s 1ms/step - loss: 1.0868
Epoch 13/200
4/4 [==============================] - 0s 1ms/st

Epoch 104/200
4/4 [==============================] - 0s 977us/step - loss: 0.1858
Epoch 105/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1818
Epoch 106/200
4/4 [==============================] - 0s 971us/step - loss: 0.1784
Epoch 107/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1751
Epoch 108/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1722
Epoch 109/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1695
Epoch 110/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1660
Epoch 111/200
4/4 [==============================] - 0s 999us/step - loss: 0.1631
Epoch 112/200
4/4 [==============================] - 0s 985us/step - loss: 0.1601
Epoch 113/200
4/4 [==============================] - 0s 983us/step - loss: 0.1571
Epoch 114/200
4/4 [==============================] - 0s 996us/step - loss: 0.1543
Epoch 115/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1518
Epoch 116/200
4/4 [=========

### Running on Google Cloud

We will use Google Cloud's ML platform, Vertex AI. To run this code on Vertex AI, we first need to:
1. Import and initialize the Vertex AI Python SDK
2. Write the ML training code to a file
3. Configure and submit a training job that runs our training code.

* Step 1. Initialize Vertex AI.

In [6]:
from helper import authenticate
CREDENTIALS, PROJECT_ID = authenticate()

In [7]:
from google.cloud import aiplatform

In [8]:
aiplatform.init(project=PROJECT_ID,
                credentials=CREDENTIALS,
                )

* Step 2. Write the ML training code to a file.

In [9]:
%%writefile task.py

# import libraries
import numpy as np
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create dataset
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, 
    centers=centers, 
    cluster_std=std,
    random_state=30)

# Create the model
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

# Train
model.fit(
    X_train,y_train,
    epochs=200
)

Overwriting task.py


In [10]:
ls

L3_Training_Models.ipynb  __pycache__/  helper.py  task.py


In [11]:
cat task.py


# import libraries
import numpy as np
from sklearn.datasets import make_blobs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create dataset
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(
    n_samples=m, 
    centers=centers, 
    cluster_std=std,
    random_state=30)

# Create the model
model = Sequential(
    [
        Dense(2, activation = 'relu',   name = "L1"),
        Dense(4, activation = 'linear', name = "L2")
    ]
)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.01),
)

# Train
model.fit(
    X_train,y_train,
    epochs=200
)


* Define the Custom Trainin Job.

> Note: Check here to find more information about [Prebuilt containers for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)

> Note: Check here to find more information about [Carbon free energy for Google Cloud regions](https://cloud.google.com/sustainability/region-carbon)

In [12]:
# Choose a location (with carbon free energy available for this example)
REGION = 'us-central1'

* Create a store bucket.

In [13]:
import random
import string

In [14]:
def generate_uuid(length: int = 8) -> str:
    return "".join(
        random.choices(
            string.ascii_lowercase + string.digits, 
            k=length))

UUID = generate_uuid()

In [15]:
# The unique identifier
UUID

'v65a2dxx'

In [16]:
from google.cloud import storage

In [17]:
storage_client = storage.Client(project = PROJECT_ID,
                                credentials = CREDENTIALS)

In [18]:
bucket_name = f'carbon-course-bucket-{UUID}'

In [19]:
bucket_name

'carbon-course-bucket-v65a2dxx'

In [20]:
bucket = storage_client.bucket(bucket_name)

In [21]:
bucket.create(location=REGION)

In [22]:
# Create the CustomTrainingJob
job = aiplatform.CustomTrainingJob(
    display_name='dlai-course-example',
    script_path='task.py',
    container_uri='us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-14.py310:latest',
    staging_bucket=f'gs://{bucket_name}',
    location=REGION,
)

In [23]:
model = job.run()

Training script copied to:
gs://carbon-course-bucket-v65a2dxx/aiplatform-2024-08-20-10-03-49.834-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://carbon-course-bucket-v65a2dxx/aiplatform-custom-training-2024-08-20-10-03-49.834
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3727316689851777024?project=362161713137
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/1619438550195896320?project=362161713137
CustomTrainingJob projects/362161713137/locations/us-central1/trainingPipelines/3727316689851777024 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/362161713137/locations/us-central1/trainingPipelines/3727316689851777024 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/362161713137/locations/us-central1/trainingPipelines/3727316689851777024 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJ

In [24]:
# Delete the bucket to clean up resources
bucket.delete(force=True)